<a href="https://colab.research.google.com/github/brody-looney/Stock-Prediction-LSTM/blob/main/FB_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Author: Brody Looney
#File: FB-LSTM.ipynb
#Purpose: Stock Predictor LSTM for COS470 Project. This model trains and tests on FB data.

In [ ]:
#upload FB_train.csv and FB_test.csv files here
from google.colab import files
uploaded = files.upload()

In [ ]:
#Importing the necessary attributes
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from keras import Sequential
from keras.layers import Dense, LSTM, Dropout

#Importing pre-split train and test sets (80/20)
train = pd.read_csv('FB_train.csv')
test = pd.read_csv('FB_test.csv')

#Dates to be used in the graph
dates = list(test['Date'])
dates = [dt.datetime.strptime(date, '%m/%d/%Y').date() for date in dates]

#Dropping the columns that won't be used in this type of predictor. This model only uses
#EMAs and the close price
train = train.drop(['Date', 'Volume', 'Open', 'High', 'Low'], axis = 1)

#Creating a scaler which will be used to scale down the train data
train_scaler = MinMaxScaler()

scaled_train = train_scaler.fit_transform(train)

#Splitting the data up into X and y sets that will be in the necessary shape for the LSTM
X_train = []
y_train = []

for i in range(1, train.shape[0]):
    X_train.append(scaled_train[i-1:i])
    y_train.append(scaled_train[i, 0])
    
X_train, y_train = np.array(X_train), np.array(y_train)

#Creating the LSTM model with multiple layers
model = Sequential()

model.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 4)))
model.add(Dropout(0.25))

model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.25))

model.add(LSTM(units = 60, return_sequences = True))
model.add(Dropout(0.25))

model.add(LSTM(units = 80))
model.add(Dropout(0.25))

model.add(Dense(units = 1))

#LSTM Compilation
model.compile(optimizer='adam', loss = 'mean_squared_error')
model.fit(X_train, y_train, epochs=10, batch_size=10)

#Creating a scaler for the test set
test_scaler = MinMaxScaler()
scaled_test = test.drop(['Date', 'Volume', 'Open', 'High', 'Low'], axis = 1)
scaled_test = test_scaler.fit_transform(scaled_test)

#Creating x and y test sets
X_test = []
y_test = []
dates_pred = []

for i in range(1, scaled_test.shape[0]):
    X_test.append(scaled_test[i-1:i])
    y_test.append(scaled_test[i, 0])
    dates_pred.append(dates[i])

X_test, y_test = np.array(X_test), np.array(y_test)

#Creating a value of the inverse of the scale so that it can be used to readjust the real price and prediction.
scale = 1/0.00816793

#Creating a prediction on the test data and scaling it 
prediction = model.predict(X_test)
prediction = prediction * scale + 145

#Getting the real price from the test set to compare the prediction to
real = test.drop(['Date', 'Volume', 'EMA10', 'EMA50', 'EMA200', 'Open', 'High', 'Low'], axis = 1)

#Printing the Real Price of FB using real
plt.figure(figsize=(14,5))
plt.plot(dates, real, color = 'red', label = 'Real FB Stock Price')
plt.gcf().autofmt_xdate()
plt.title('FB Stock Price')
plt.xlabel('Date')
plt.ylabel('FB Stock Price')

#Printing the prediction of the price of AAPL by using values of prediction
plt.plot(dates_pred, prediction, color = 'blue', label = 'Predicted FB Stock Price')
plt.gcf().autofmt_xdate()
plt.title('FB Stock Price Prediction')
plt.xlabel('Date')
plt.ylabel('FB Stock Price')
plt.legend()
plt.show()

In [ ]:
train_scaler.scale_

array([0.00607497, 0.00607497, 0.00607497, 0.00592277])

In [ ]:
test_scaler.scale_

array([0.00918695, 0.00918695, 0.00918695, 0.00816793])